### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 30 апреля 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 30 апреля, -4 балла после 06:00 7 мая, -6 баллов после 06:00 14 мая, -8 баллов после 06:00 21 мая

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
#%load_ext pycodestyle_magic

In [265]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = []
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise
        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return  1 - np.sum((l_c ** 2) / (l_s * (l_s + r_s)) + (r_c ** 2) / (r_s * (l_s + r_s)), axis=1)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return -np.sum((l_c / l_s).log2() * l_c / (l_s + r_s) + (r_c / r_s).log2() * r_c / (l_s + r_s), axis=1)

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - np.max(l_c / (l_s + r_s), axis=1) - np.max(r_c / (l_s + r_s), axis=1)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.sqrt(feature_ids.shape[0]))])
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.log2(feature_ids.shape[0]))])

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        #print(feature_id)
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        #self.feature_importances_ = []
        xsort, ysort = self.__sort_samples(x, y)
        classnum = self.num_class
        tocut = np.int(self.min_samples_split / 2 - 1)
        if tocut == 0:
            ysplit = ysort
        else:
            ysplit = ysort[tocut:-tocut]
        newind = np.where(ysplit[:-1] != ysplit[1:])[0] + (tocut + 1)
        if len(newind) == 0:
            return np.inf, None
        equal = newind - np.append(np.array([tocut]), newind[:-1])
        changemat = np.zeros((newind.shape[0], classnum))
        changemat[np.arange(newind.shape[0]), ysort[newind - 1]] = 1
        classnummat = changemat * equal.reshape(-1, 1)
        classnummat[0] = classnummat[0] + np.bincount(ysort[:tocut], minlength=classnum)
        #print(classnummat)
        lclassnum = np.cumsum(classnummat, axis=0)
        rclassnum = np.bincount(ysort, minlength=classnum) - lclassnum
        lsizes = newind.reshape(lclassnum.shape[0], 1)
        rsizes = ysort.shape[0] - lsizes
        imp = self.G_function(lclassnum, lsizes, rclassnum, rsizes)
        ind = np.argmin(imp)
        #self.feature_importances_.append(ind)
        return imp[ind], (xsort[lsizes[ind][0]-1] + xsort[lsizes[ind][0]]) / 2.0

    def __fit_node(self, x, y, node_id, depth):
        if depth == self.max_depth or self.min_samples_split >= x.shape[0] or len(np.unique(y)) == 1:
            self.tree[node_id] = (self.__class__.LEAF_TYPE,
                                  np.bincount(y).argmax(), np.bincount(y) / y.size)
            return
        threshold = np.array([self.__find_threshold(x[:, i], y) for i in self.get_feature_ids(x.shape[1])])
        feature = threshold[:, 0].argmin()
        best_threshold = threshold[feature, 1]
        self.tree[node_id] = (self.__class__.NON_LEAF_TYPE, feature, best_threshold)

        X_l, X_r, y_l, y_r = self.__div_samples(x, y, feature, best_threshold)
        if X_l.shape[0] == 0 or X_r.shape[0] == 0:
            self.tree[node_id] = (self.__class__.LEAF_TYPE, np.bincount(y).argmax(), np.bincount(y).astype(float) / y.size)
            return

        self.__fit_node(X_l, y_l, 2 * node_id + 1, depth + 1)
        self.__fit_node(X_r, y_r, 2 * node_id + 2, depth + 1)
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                self.feature_importances_.append(feature_id)
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [266]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [267]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [268]:
%time clf.fit(X_train, y_train)

Wall time: 1 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [269]:
%time my_clf.fit(X_train, y_train)

Wall time: 10 ms


## Проверка качества работы на wine

In [270]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9474747474747475

In [271]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.9474747474747475

## Подготовка данных Speed Dating Data 

In [272]:
df = pd.read_csv('./https___raw.githubusercontent.com_alkhamush_spheremailru_master_2019-02_hw3_speed-dating-experiment_Speed%20Dating%20Data.csv', sep=',', encoding="ISO-8859-1")
df = df.iloc[:, :97]
df = df.drop(['id'], axis=1)
df = df.drop(['idg'], axis=1)
df.drop_duplicates(subset=['iid']).gender.value_counts()
df = df.drop(['condtn'], axis=1)
df = df.drop(['round'], axis=1)
df = df.drop(['position', 'positin1'], axis=1)
df = df.drop(['order'], axis=1)
df = df.drop(['partner'], axis=1)
df = df.drop(['age_o', 'race_o', 'pf_o_att','pf_o_sin', 'pf_o_int','pf_o_fun', 'pf_o_amb', 'pf_o_sha','dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o','amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o'],axis=1)
df = df.dropna(subset=['age'])
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)
df = df.dropna(subset=['imprelig', 'imprace'])
df = df.drop(['from', 'zipcode'], axis=1)
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
df = df.dropna(subset=['date'])
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
df = df.drop(['career'], axis=1)
df = df.drop(['sports', 'tvsports', 'exercise','dining', 'museums', 'art', 'hiking','gaming', 'clubbing', 'reading', 'tv','theater', 'movies', 'concerts','music', 'shopping', 'yoga'], axis=1)
df = df.drop(['expnum'], axis=1)
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
df = df.drop(['field'], axis=1)
pd.get_dummies(df, columns=['field_cd'],prefix='field_cd', prefix_sep='=')
df = df.drop(['undergra'], axis=1)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1','fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1','intel1_1', 'fun1_1','amb1_1', 'shar1_1']] = (df.loc[:, ['attr1_1', 'sinc1_1','intel1_1', 'fun1_1','amb1_1', 'shar1_1']].T / df.loc[:, 'temp_totalsum'].T).T * 100
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1','intel2_1', 'fun2_1','amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1','intel2_1', 'fun2_1','amb2_1', 'shar2_1']] = (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1','amb2_1', 'shar2_1']].T / df.loc[:, 'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)
for i in [4, 5]:
    todrop = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),'intel{}_1'.format(i), 'fun{}_1'.format(i), 'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    if i != 4:
        todrop.remove('shar{}_1'.format(i))
    df = df.drop(todrop, axis=1)
df = df.drop(['wave'], axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid']) .drop(['gender'], axis=1) .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid']) .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1) .dropna()
df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'),on='pid',how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы на Speed Dating Data 

In [273]:
%time clf.fit(X_train, y_train)

Wall time: 72 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [274]:
%time my_clf.fit(X_train, y_train)

Wall time: 564 ms


## Проверка качества работы на Speed Dating Data

In [275]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5587583377148219

In [276]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5211492418196328

## Задание 3

In [277]:
df_pair.columns[np.argsort(-clf.feature_importances_)][:10]

Index(['match', 'imprelig_f', 'fun1_1', 'samerace', 'attr1_1', 'intel2_1_f',
       'intel3_1', 'fun1_1_f', 'attr1_1_f', 'shar2_1_f'],
      dtype='object')

In [280]:
df_pair.columns[(my_clf.feature_importances_)[:10]]

Index(['samerace', 'goal', 'amb1_1', 'sinc1_1', 'date', 'attr3_1_f', 'sinc1_1',
       'amb1_1', 'sinc1_1', 'goal'],
      dtype='object')

## Задание 4

In [288]:
grid = {
    'max_depth': [5, 50, 100],
    'min_samples_leaf': [4, 10, 30],
    'criterion': ['gini', 'entropy',],
    'n_estimators': [2,6,10]
}
clf = RandomizedSearchCV(RandomForestClassifier(),
                          param_distributions=grid)
clf.fit(X_train, y_train)
clf.best_params_

D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


{'n_estimators': 10,
 'min_samples_leaf': 10,
 'max_depth': 50,
 'criterion': 'entropy'}

## Задание 5